# AADT Training

## Imports

In [607]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

## Global Variables

In [608]:
ROOT_DIR_PATH = os.path.abspath('..')

IMG_SIZE = 1024
VEHICLE_DETECTION_COUNT_PATH = os.path.join(ROOT_DIR_PATH, 'data/vehicle_counts_detection.csv')
AADT_PROCESSED_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/aadt_processed.csv')

NN_MODEL_PATH = os.path.join(ROOT_DIR_PATH, "models/nn_aadt_model.pth")

## True Count Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Neural Network Model

In [609]:
class CustomDataset(Dataset):
    def __init__(self):
        self.labels = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['aadt'].values.astype('float32'))
        self.vehicle_count = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['total_volume'].values.astype('float32')).unsqueeze(1) # Training
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.hour = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['hour'].values.astype('float32')).unsqueeze(1)
        self.avg_mph = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['avg_mph'].values.astype('float32')).unsqueeze(1)
        self.day = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['day'].values.astype('float32')).unsqueeze(1)
        self.month = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['month'].values.astype('float32')).unsqueeze(1)
        self.small_vehicle = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['0-520cm'].values.astype('float32')).unsqueeze(1)
        self.mid_vehicle = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['521-660cm'].values.astype('float32')).unsqueeze(1)
        self.large_vehicle = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['661-1160cm'].values.astype('float32')).unsqueeze(1)
        self.very_large_vehicle = torch.tensor(pd.read_csv(AADT_PROCESSED_PATH)['1160+cm'].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.vehicle_count, self.small_vehicle, self.mid_vehicle, self.large_vehicle, self.very_large_vehicle, self.avg_mph, self.day, self.month, self.hour), dim=-1)
        self.y = self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

custom_data = CustomDataset()
train_split = 0.8
train_data, val_data = random_split(custom_data, [train_split, 1-train_split])

In [610]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9, 9),
            nn.Linear(9,20),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(20,20),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(20,1),
            nn.LeakyReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [611]:
nn_model = NeuralNetwork()
nn_model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=9, out_features=9, bias=True)
    (1): Linear(in_features=9, out_features=20, bias=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=20, out_features=1, bias=True)
    (8): LeakyReLU(negative_slope=0.01)
  )
)

## EarlyStopping

In [612]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [613]:
early_stopper = EarlyStopper(patience=3, min_delta=10)

## Weights Initialisation

In [614]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

nn_model.apply(init_weights)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=9, out_features=9, bias=True)
    (1): Linear(in_features=9, out_features=20, bias=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=20, out_features=1, bias=True)
    (8): LeakyReLU(negative_slope=0.01)
  )
)

## HyperParameters

In [615]:
learning_rate = 1e-1
batch_size = 1
epochs = 1
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

MAPE = MeanAbsolutePercentageError()

## DataLoaders

In [616]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

In [617]:
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

## Training & Validation

In [618]:
def run_epoch(ep_id, action, loader, model, optimizer, criterion):
    losses = [] # Keep list of accuracies to track progress
    is_training = action == "train" # True when action == "train", else False 

    # Looping over all batches
    for batch_idx, batch in enumerate(loader): 
        x, y = batch

        # Resetting the optimizer gradients
        optimizer.zero_grad()

        # Setting model to train or test
        with torch.set_grad_enabled(is_training):
            
            # Feed batch to model
            logits = nn_model(x).squeeze(1)
            print("logits: {}".format(logits))

            # Calculate the loss based on predictions and real labels
            loss = criterion(logits, y)
            mape_loss = MAPE(logits, y)
            print("MAPE loss: {}".format(mape_loss))

            # If training, perform backprop and update weights
            if is_training:
                loss.backward()
                optimizer.step()

            # Append current batch accuracy
            losses.append(mape_loss.detach().numpy())

            # Print some stats every 50th batch 
            if batch_idx % 50 == 0:
                print(f"{action.capitalize()}ing, Epoch: {ep_id+1}, Batch {batch_idx}: Loss = {loss.item()}")

        if not is_training:
            if early_stopper.early_stop(mape_loss.detach().numpy()):             
                break
                    
    # Return accuracies to main loop                 
    return losses

In [619]:
def main(epochs, train_dl, val_dl, model, optimizer, criterion):

    # Keep lists of accuracies to track performance on train and test sets
    train_losses = []
    val_losses = []

    # Looping over epochs
    for epoch in range(epochs):
        
        # Looping over train set and training
        train_loss = run_epoch(epoch, "train", train_dl, model, optimizer, criterion)

        # Looping over test set
        val_loss = run_epoch(epoch, "val", val_dl, model, optimizer, criterion) 

        # Collecting stats
        train_losses += train_loss
        val_losses += val_loss         
            
    return train_losses, val_losses

In [620]:
train_losses, val_losses = main(epochs=epochs, train_dl=train_dataloader, val_dl=val_dataloader, model=nn_model, optimizer=optimizer, criterion=loss_fn)

logits: tensor([-0.1393], grad_fn=<SqueezeBackward1>)
MAPE loss: 1.0000219345092773
Training, Epoch: 1, Batch 0: Loss = 40134788.0
logits: tensor([152.4614], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9916427135467529
logits: tensor([812.4283], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9554659724235535
logits: tensor([107.7175], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9940953850746155
logits: tensor([3304.3904], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.8188666105270386
logits: tensor([709.1791], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.8880549073219299
logits: tensor([5955.5957], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.059899527579545975
logits: tensor([3301.3276], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.47888004779815674
logits: tensor([39050.9844], grad_fn=<SqueezeBackward1>)
MAPE loss: 1.1406179666519165
logits: tensor([9956.1035], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.4542464315891266
logits: tensor([7159.0039], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.13006041944026947
logits: 

### Loss Curve Plot

In [621]:
px.line(train_losses)

In [622]:
px.line(val_losses)

### Random val samples

In [623]:
preds = []
ground_truth = []
vehicle_counts = []


for i in range(100):
    vehicle_count = 0
    random_idx = np.random.randint(0,len(val_data))
    x, y = val_data[random_idx]
    vehicle_count = x[0]
    vehicle_counts.append(vehicle_count)
    ground_truth.append(float(y))
    pred_y = float(nn_model(x)[0])
    preds.append(pred_y)

df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})

px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'])

## Save Model

In [624]:
torch.save(nn_model.state_dict(), NN_MODEL_PATH)